# Process antibody hashtag reads and assign to cells
(Pre-processing step for Scanpy)

In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import os
import csv

In [2]:
### Self defined functions for identifying cell antibody hashtags
def findAb(ftTsv):
    AbDict = {}
    with open(ftTsv, "r") as fin:
        rfin = csv.reader(fin, delimiter="\t")
        rowN = 0
        for row in rfin:
            rowN += 1
            if row[2] == "Antibody Capture":
                AbDict[rowN] = row[0]
    return(AbDict)

def cellAbReads(ftTsv, mtxMtx):
    outName = "Cells_hashTags.csv"
    abDict = findAb(ftTsv)
    with open(mtxMtx, "r") as fin:
        with open(outName, "w") as fout:
            rfin = csv.reader(fin, delimiter=" ")
            wfout = csv.writer(fout, delimiter=",")
            wfout.writerow(["Cell"] + list(abDict.values()))
            
            next(rfin)
            next(rfin)
            next(rfin)
            step = 1
            outRow = [1] + [0 for x in abDict]
            for row in rfin:
                cellN = int(row[1])
                if cellN != step:
                    wfout.writerow(outRow)
                    step = cellN
                    outRow = [row[1]] + [0 for x in abDict]
                if int(row[0]) in list(abDict.keys()):
                    abIdx = list(abDict.keys()).index(int(row[0])) + 1
                    outRow[abIdx] = int(row[2])
            wfout.writerow(outRow)

def cellType(cellTag):
    outfile = "Cells_Type.csv"
    with open(cellTag, "r") as fin:
        with open(outfile, "w") as fout:
            rfin = csv.reader(fin, delimiter=",")
            wfout = csv.writer(fout, delimiter=",")
            all_types = next(rfin)[1:]
            cell_types = []
            for row in rfin:
                row_nu = [int(x) for x in row[1:]]
                rowMax = max(row_nu)
                rowSum = sum(row_nu)
                row_type = all_types[row_nu.index(rowMax)]
                if rowMax < float(rowSum)*0.7:
                    #print('Error in cell %s, maximum hashtag reads less than 70 percent...' %row[0])
                    #print(row)
                    row_type = "Ambigous"
                    
                wfout.writerow([row_type])
                cell_types.append(row_type)
    types_set = list(set(cell_types))
    print("Total cell number: %s" %str(len(cell_types)))
    print("-------------------")
    print("Ambigous percentage: %s" %str(float(cell_types.count("Ambigous"))*100/len(cell_types)))
    print("-------------------")
    for i in types_set:
        if i != "Ambigous":
            print("%s percentage: %s" %(i, str(float(cell_types.count(i))*100/len(cell_types))))
    return(cell_types)

In [4]:
code_dir = os.getcwd()
base_dir = code_dir.replace("/codes_local","")
wk_dir = base_dir + "1_Scanpy"

'/media/pipkin/ROCKET-PRO/CD8_DEV_SC/Acute_Chronic/codes_local'

In [3]:
wk_dir = "/Volumes/Yolanda/Exp391_Acute-Chronic_SC/1_Scanpy"
os.chdir(wk_dir)

In [5]:
ft_tsv = "/Volumes/Yolanda/Exp391_Acute-Chronic_SC/1_0_cellranger_outs/filtered_feature_bc_matrix/features.tsv"
mtx_mtx = "/Volumes/Yolanda/Exp391_Acute-Chronic_SC/1_0_cellranger_outs/filtered_feature_bc_matrix/matrix.mtx"
cellAbReads(ft_tsv, mtx_mtx)
Exp391_celltypes = cellType("Cells_hashTags.csv")

Total cell number: 18353
-------------------
Ambigous percentage: 11.267912602844222
-------------------
NWT percentage: 5.094534953413611
A8T percentage: 9.268239524873318
A8P percentage: 7.110554132839318
A5T percentage: 19.239361412303165
A5P percentage: 20.53615212771754
C5P percentage: 15.88296191358361
C8P percentage: 5.900942625183894
NP14B percentage: 5.699340707241323
